In [56]:
import torchvision.utils as vutils


In [55]:
# 数据加载器的定义

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class FloorPlanDataset(Dataset):
    def __init__(self, black_images_dir, color_images_dir, transform=None):
        self.black_images_dir = black_images_dir
        self.color_images_dir = color_images_dir
        self.transform = transform
        self.black_images = sorted(os.listdir(black_images_dir))
        self.color_images = sorted(os.listdir(color_images_dir))
    
    def __len__(self):
        return len(self.black_images)
    
    def __getitem__(self, idx):
        black_image_path = os.path.join(self.black_images_dir, self.black_images[idx])
        color_image_path = os.path.join(self.color_images_dir, self.color_images[idx])
        
        black_image = Image.open(black_image_path).convert('RGB')
        color_image = Image.open(color_image_path).convert('RGB')
        
        if self.transform:
            black_image = self.transform(black_image)
            color_image = self.transform(color_image)
        
        return black_image, color_image

# 定义数据增强或变换
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图像大小
    transforms.ToTensor()           # 转换为张量
])

# 初始化数据集和数据加载器
black_images_dir = '../Dataset/A'  # 黑色轮廓图所在的目录
color_images_dir = '../Dataset/B'  # 彩色照片所在的目录

dataset = FloorPlanDataset(black_images_dir, color_images_dir, transform)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)



In [50]:
# 定义生成器
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(input_nc, ngf, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.Conv2d(ngf, ngf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            nn.Conv2d(ngf * 2, ngf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            nn.Conv2d(ngf * 4, ngf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(ngf, output_nc, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
    
    def forward(self, input):
        return self.main(input)

# 初始化生成器
generator = Generator(input_nc=3, output_nc=3).to(device)


In [51]:
# 定义判别器
class Discriminator(nn.Module):
    def __init__(self, input_nc, ndf=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(input_nc, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.main(input)

# 初始化判别器
discriminator = Discriminator(input_nc=6).to(device)  # 3 channels for black images + 3 channels for color images


In [52]:
# 定义损失函数
# 定义GAN损失函数
criterion_GAN = torch.nn.BCEWithLogitsLoss()

# 定义L1损失，用于生成图像与目标图像之间的像素差异
criterion_L1 = torch.nn.L1Loss()


In [53]:
# 定义优化器
# 定义生成器和判别器的优化器
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [60]:
# 设定输出路径
output_dir = 'generated_images/2'


In [61]:
# 训练设置
num_epochs = 100

for epoch in range(num_epochs):
    for i, (black_images, color_images) in enumerate(dataloader):
        # 将数据移动到设备
        black_images = black_images.to(device)
        color_images = color_images.to(device)

        # ---------------------------
        # 训练判别器
        # ---------------------------
        fake_images = generator(black_images)
        real_input = torch.cat((black_images, color_images), dim=1)
        fake_input = torch.cat((black_images, fake_images.detach()), dim=1)

        output_real = discriminator(real_input)
        output_fake = discriminator(fake_input)

        real_labels = torch.ones_like(output_real).to(device)
        fake_labels = torch.zeros_like(output_fake).to(device)

        loss_D_real = criterion_GAN(output_real, real_labels)
        loss_D_fake = criterion_GAN(output_fake, fake_labels)
        loss_D = (loss_D_real + loss_D_fake) / 2

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # ---------------------------
        # 训练生成器
        # ---------------------------
        output_fake = discriminator(fake_input)
        loss_G_GAN = criterion_GAN(output_fake, real_labels)
        loss_G_L1 = criterion_L1(fake_images, color_images)
        loss_G = loss_G_GAN + 100 * loss_G_L1

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        # 每个批次打印一次损失
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], "
              f"Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

    # ---------------------------
    # 保存拼接后的图像
    # ---------------------------
    with torch.no_grad():
        fake_images = generator(black_images)
        combined_output = torch.cat((black_images, fake_images), dim=3)

        # 保存图像到指定目录
        vutils.save_image(combined_output, os.path.join(output_dir, f"epoch_{epoch+1}.png"), normalize=True)
        print(f"Saved combined images for epoch {epoch+1}")

Epoch [1/100], Step [1/13], Loss_D: 0.5250, Loss_G: 14.0912
Epoch [1/100], Step [2/13], Loss_D: 0.5231, Loss_G: 14.1824
Epoch [1/100], Step [3/13], Loss_D: 0.5208, Loss_G: 13.6077
Epoch [1/100], Step [4/13], Loss_D: 0.5180, Loss_G: 12.8726
Epoch [1/100], Step [5/13], Loss_D: 0.5158, Loss_G: 12.7541
Epoch [1/100], Step [6/13], Loss_D: 0.5148, Loss_G: 13.8157
Epoch [1/100], Step [7/13], Loss_D: 0.5135, Loss_G: 12.9290
Epoch [1/100], Step [8/13], Loss_D: 0.5128, Loss_G: 13.3654
Epoch [1/100], Step [9/13], Loss_D: 0.5108, Loss_G: 12.5836
Epoch [1/100], Step [10/13], Loss_D: 0.5120, Loss_G: 13.7950
Epoch [1/100], Step [11/13], Loss_D: 0.5140, Loss_G: 9.3903
Epoch [1/100], Step [12/13], Loss_D: 0.5108, Loss_G: 10.7620
Epoch [1/100], Step [13/13], Loss_D: 0.5121, Loss_G: 13.0580
Saved combined images for epoch 1
Epoch [2/100], Step [1/13], Loss_D: 0.5138, Loss_G: 9.9873
Epoch [2/100], Step [2/13], Loss_D: 0.5114, Loss_G: 10.3280
Epoch [2/100], Step [3/13], Loss_D: 0.5097, Loss_G: 12.5634
Epoc